In [1]:
import time

log_file = 'log.txt'
# clean the log
f = open(log_file, 'w')
f.close()

def find_repo(term, day):
    import requests
    import math
    from datetime import datetime, timedelta
    from dateutil.relativedelta import relativedelta
    import csv
    import pandas as pd 
    import json
    import os
    
    token=os.environ['GITHUBTOKEN']
    URL = f'https://api.github.com/search/repositories?q={term}+created:SINCE..UNTIL&per_page=100'
    HEADERS = {'Authorization':f'{token}'}
    file_list=[]
    since = datetime.today() - relativedelta(days= day)  # Start fetching repo created {day} days ago
    until = since + timedelta(hours=4) # dividing the total No.of repo into segments of 4 hours each
    fetch_time=f'{until}'.replace(":","_")
    filename = f'{term}_{day}_{fetch_time}.json'
    
    # writing json file
    f= open(filename,'w',encoding='utf-8')
    f.close()
        
    while until < datetime.today():
        day_url = URL.replace('SINCE', since.strftime('%Y-%m-%dT%H:%M:%SZ')).replace('UNTIL', until.strftime('%Y-%m-%dT%H:%M:%SZ'))
        response = requests.get(day_url, headers=HEADERS)
        
        #writing log file:
        f = open(log_file, 'a')
        f.write(day_url+'\t'+str(response.status_code)+'\n')
        f.close()
        repo_request=response.json()
        
        #saving json files:
        converted_to_string=json.dumps(repo_request)
        f=open(filename,'a',encoding='utf-8')
        f.write(converted_to_string + '\n')
        f.close()
        
        # Update dates for the next search
        since = until #move start-date and end-date up 12hours
        until += timedelta(hours=4)
        time.sleep(2)
    file_list.append(filename)
          
    return filename

In [2]:
#try the function
filename = find_repo("education",1)

In [3]:
filename

'education_1_2021-10-08 15_54_58.273192.json'

In [4]:
#run the below 
#copy paste the json file below

f=open(filename,'r',encoding='utf-8')
content=f.readlines()
content

['{"total_count": 12, "incomplete_results": false, "items": [{"id": 415031626, "node_id": "R_kgDOGLzhSg", "name": "Education", "full_name": "TaranpreetSingh2000/Education", "private": false, "owner": {"login": "TaranpreetSingh2000", "id": 88654743, "node_id": "MDQ6VXNlcjg4NjU0NzQz", "avatar_url": "https://avatars.githubusercontent.com/u/88654743?v=4", "gravatar_id": "", "url": "https://api.github.com/users/TaranpreetSingh2000", "html_url": "https://github.com/TaranpreetSingh2000", "followers_url": "https://api.github.com/users/TaranpreetSingh2000/followers", "following_url": "https://api.github.com/users/TaranpreetSingh2000/following{/other_user}", "gists_url": "https://api.github.com/users/TaranpreetSingh2000/gists{/gist_id}", "starred_url": "https://api.github.com/users/TaranpreetSingh2000/starred{/owner}{/repo}", "subscriptions_url": "https://api.github.com/users/TaranpreetSingh2000/subscriptions", "organizations_url": "https://api.github.com/users/TaranpreetSingh2000/orgs", "repos_

In [5]:
len(content)

6

In [6]:
#taking list of repo

repo_list = []
import json 

for item in content:
    jsonobj=json.loads(item)
    repo_list.extend(jsonobj["items"])
    
len(repo_list)

49

In [15]:
dt =[]

for repo in repo_list:
    id = repo.get("id")
    name =repo.get("name")
    url = repo.get("html_url")
    created = repo.get("created_at")
    stars = repo.get("stargazers_count")
    watch = repo.get("watchers_count")
    language = repo.get("language")
    forks = repo.get("forks_count")

    import requests
    from bs4 import BeautifulSoup
    readme=requests.get(url)
    soup =BeautifulSoup(readme.text,"html.parser")
    readme=soup.find(class_="markdown-body entry-content container-lg")
    if readme == None:
        readme =("")
    else: 
        readme=soup.find(class_="markdown-body entry-content container-lg").get_text().replace('\n'," ")

    dt.append({"id": id,
                 "name": name,
                 "url": url,
                 "created": created,
                 "stars": stars,
                 "watch": watch,
                 "language": language,
                 "forks": forks,
                 "readme":readme})

In [16]:
dt[0]

{'id': 415031626,
 'name': 'Education',
 'url': 'https://github.com/TaranpreetSingh2000/Education',
 'created': '2021-10-08T15:09:18Z',
 'stars': 0,
 'watch': 0,
 'language': 'HTML',
 'forks': 0,
 'readme': 'The-Sparks-Foundation\nResponsive Payment Gateway Integration website\n'}

In [18]:
#the problem here is that readme contains special characters, it can't be written in the csv file
#if we remove the readme, then file can be saved and the following works well

import csv
import pandas as pd 

#Writing data into .csv file and returning the table:
with open("dt.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["id", "name", "url", "language", "created", "stars", "watch", "forks", "readme"])
    
    for item in dt:
        writer.writerow([item['id'], item['name'], item['url'], item['language'], item['created'], item['stars'], item['watch'], item['forks'], item['readme']])

data = pd.read_csv("dt.csv", delimiter= ";")


UnicodeEncodeError: 'charmap' codec can't encode characters in position 1422-1424: character maps to <undefined>

In [13]:
data

,id,name,url,language,created,stars,watch,forks
0,415031626,Education,https://github.com/TaranpreetSingh2000/Education,HTML,2021-10-08T15:09:18Z,0,0,0
1,414976733,education,https://github.com/madhupeta29/education,TypeScript,2021-10-08T12:23:47Z,0,0,0
2,415019593,EducationTasks,https://github.com/BerkovichPeter/EducationTasks,Java,2021-10-08T14:32:39Z,0,0,0
3,415043444,ecolocigal--education,https://github.com/PeleWkurzony/ecolocigal--ed...,HTML,2021-10-08T15:45:57Z,0,0,0
4,414968108,Afrofem-education,https://github.com/EVELYNAJUNA/Afrofem-education,NaN,2021-10-08T11:55:38Z,0,0,0
5,415041222,Education-Website,https://github.com/Leodiwas/Education-Website,NaN,2021-10-08T15:38:57Z,0,0,0
6,414972110,Education-website,https://github.com/RahulSinghAswal/Education-w...,HTML,2021-10-08T12:08:35Z,0,0,0
7,414969588,cubic-workshop,https://github.com/ElkaDerm/cubic-workshop,Handlebars,2021-10-08T12:00:46Z,0,0,0
8,415009868,blitz_js_tutorial_1,https://github.com/sakitayu/blitz_js_tutorial_1,TypeScript,2021-10-08T14:04:52Z,0,0,0
9,415033019,github-education-teacher-training-prof-castellina,https://github.com/Campus-Advisors/github-educ...,NaN,2021-10-08T15:13:23Z,0,0,0
